# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [3]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.24,72,7,3.67,clear sky
1,1,Kapaa,US,22.0752,-159.3190,75.18,85,0,6.91,clear sky
2,2,Albany,US,42.6001,-73.9662,46.31,81,100,3.53,light rain
3,3,Matagami,CA,49.7502,-77.6328,28.51,74,100,9.22,overcast clouds
4,4,Te Anau,NZ,-45.4167,167.7167,44.96,94,100,1.39,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [4]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [5]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]


# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.24,72,7,3.67,clear sky
1,1,Kapaa,US,22.0752,-159.3190,75.18,85,0,6.91,clear sky
5,5,Saint-Philippe,RE,-21.3585,55.7679,72.99,71,1,7.74,clear sky
9,9,Atuona,PF,-9.8000,-139.0333,77.79,70,7,14.67,clear sky
10,10,Mar Del Plata,AR,-38.0023,-57.5575,77.83,88,0,9.22,clear sky
11,11,Bathsheba,BB,13.2167,-59.5167,84.81,70,40,24.16,scattered clouds
19,19,Avarua,CK,-21.2078,-159.7750,78.85,65,40,9.22,scattered clouds
20,20,Arequipa,PE,-16.3989,-71.5350,71.67,37,20,12.66,few clouds
21,21,Butaritari,KI,3.0707,172.7902,80.76,76,12,11.52,light rain
22,22,Ipixuna,BR,-1.7625,-48.8043,86.58,59,54,4.88,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [8]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.count()

# Display sample data
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.24,72,7,3.67,clear sky
1,1,Kapaa,US,22.0752,-159.3190,75.18,85,0,6.91,clear sky
5,5,Saint-Philippe,RE,-21.3585,55.7679,72.99,71,1,7.74,clear sky
9,9,Atuona,PF,-9.8000,-139.0333,77.79,70,7,14.67,clear sky
10,10,Mar Del Plata,AR,-38.0023,-57.5575,77.83,88,0,9.22,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng
0,Rikitea,PF,75.24,clear sky,-23.1203,-134.9692
1,Kapaa,US,75.18,clear sky,22.0752,-159.3190
5,Saint-Philippe,RE,72.99,clear sky,-21.3585,55.7679
9,Atuona,PF,77.79,clear sky,-9.8000,-139.0333
10,Mar Del Plata,AR,77.83,clear sky,-38.0023,-57.5575
11,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167
19,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750
20,Arequipa,PE,71.67,few clouds,-16.3989,-71.5350
21,Butaritari,KI,80.76,light rain,3.0707,172.7902
22,Ipixuna,BR,86.58,broken clouds,-1.7625,-48.8043


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [10]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.24,clear sky,-23.1203,-134.9692,
1,Kapaa,US,75.18,clear sky,22.0752,-159.3190,
5,Saint-Philippe,RE,72.99,clear sky,-21.3585,55.7679,
9,Atuona,PF,77.79,clear sky,-9.8000,-139.0333,
10,Mar Del Plata,AR,77.83,clear sky,-38.0023,-57.5575,
11,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167,
19,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,
20,Arequipa,PE,71.67,few clouds,-16.3989,-71.5350,
21,Butaritari,KI,80.76,light rain,3.0707,172.7902,
22,Ipixuna,BR,86.58,broken clouds,-1.7625,-48.8043,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [12]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [13]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Rikitea - nearest hotel: Chez Bianca & Benoit
Kapaa - nearest hotel: Pono Kai Resort
Saint-Philippe - nearest hotel: Le Baril
Atuona - nearest hotel: Pearl Resort
Mar Del Plata - nearest hotel: Nuevo Ostende
Bathsheba - nearest hotel: Atlantis Hotel
Avarua - nearest hotel: Paradise Inn
Arequipa - nearest hotel: Casa Andina
Butaritari - nearest hotel: No hotel found
Ipixuna - nearest hotel: No hotel found
Obo - nearest hotel: No hotel found
Lazaro Cardenas - nearest hotel: Hotel Sol del Pacífico
Takoradi - nearest hotel: Hillcrest
Namibe - nearest hotel: Hotel Chik Chik Namibe
Fare - nearest hotel: hôtel Maitai Lapita Village
Sur - nearest hotel: Sur Hotel
Padang - nearest hotel: losmen Surya
Sinnamary - nearest hotel: Hôtel du fleuve
Labuhan - nearest hotel: No hotel found
Puebla - nearest hotel: No hotel found
Arraial Do Cabo - nearest hotel: No hotel found
Sao Francisco Do Sul - nearest hotel: Hotel Zibamba
Mandera - nearest hotel: Kornesh Hotel and Lodging
Ati 

Viedma - nearest hotel: Níjar
Morrope - nearest hotel: No hotel found
Rawson - nearest hotel: Hotel Deportivo
Namatanai - nearest hotel: Namatanai Hotel
Edeia - nearest hotel: Hotel Roma
Araceli - nearest hotel: ARPA Lodge
Paratinga - nearest hotel: No hotel found
Banjarmasin - nearest hotel: Hotel Grand Mentari
Fomboni - nearest hotel: Mohéli La Plage
Eyl - nearest hotel: فندق ايل دابي
Vanimo - nearest hotel: CBC Guesthouse
Benguela - nearest hotel: Hotel Moibela
Raymondville - nearest hotel: No hotel found
Puerto Leguizamo - nearest hotel: Hotel Girasoles
Wajid - nearest hotel: No hotel found
Tautira - nearest hotel: No hotel found
Waingapu - nearest hotel: HOTEL SANDLEWOOD
Hatillo - nearest hotel: No hotel found
Navirai - nearest hotel: Hotel 2 Gaúchos
Kaabong - nearest hotel: No hotel found
Aswan - nearest hotel: Yaseen hotel
Sao Miguel Do Iguacu - nearest hotel: No hotel found
Paka - nearest hotel: Paka Inn
Mugango - nearest hotel: No hotel found
Icod De Los Vinos - nearest hotel:

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.24,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
1,Kapaa,US,75.18,clear sky,22.0752,-159.3190,Pono Kai Resort
5,Saint-Philippe,RE,72.99,clear sky,-21.3585,55.7679,Le Baril
9,Atuona,PF,77.79,clear sky,-9.8000,-139.0333,Pearl Resort
10,Mar Del Plata,AR,77.83,clear sky,-38.0023,-57.5575,Nuevo Ostende
...,...,...,...,...,...,...,...
669,Conakry,GN,81.75,overcast clouds,9.5380,-13.6773,Hotel del Fanta (sleazy but rooms for 100k or ...
673,Sao Gabriel Da Cachoeira,BR,89.40,few clouds,-0.1303,-67.0892,Casa de Hóspedes do CFRN/5ºBIS
677,Itaituba,BR,87.96,scattered clouds,-4.2761,-55.9836,TG Hotel
678,Pontianak,ID,73.38,scattered clouds,-0.0333,109.3333,Golden Tulip Hotel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [21]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.24,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
1,Kapaa,US,75.18,clear sky,22.0752,-159.3190,Pono Kai Resort
5,Saint-Philippe,RE,72.99,clear sky,-21.3585,55.7679,Le Baril
9,Atuona,PF,77.79,clear sky,-9.8000,-139.0333,Pearl Resort
10,Mar Del Plata,AR,77.83,clear sky,-38.0023,-57.5575,Nuevo Ostende
11,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167,Atlantis Hotel
19,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
20,Arequipa,PE,71.67,few clouds,-16.3989,-71.5350,Casa Andina
30,Lazaro Cardenas,MX,84.49,clear sky,17.9583,-102.2000,Hotel Sol del Pacífico
31,Takoradi,GH,79.48,clear sky,4.8845,-1.7554,Hillcrest


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [22]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [24]:
# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Description"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Description)